# EU climate change mitigation targets compromise forest ecosystem services and biodiversity

## Germany


Above the code cells, there are short instructions how the users can modify the codes in the cells. If there are no instructions, no changes should be needed for the cell by default.

A detailed description is provided in the README.md.

## Basic definitions
Simulated forest data - name and climate scenario

Specify policy scenario:

- "FS" -  National Forest Strategy
- "BDS" - Biodiversity Strategy
- "BES" - Bioeconomy Strategy

In [1]:
scenario ="NFS"

## Specify "if" and "which" GLOBIOM scenario is used

**ATTENTION**: each scenario relates to specific CC simulation!<br>
Three scenarios define the perdiodic future demand for log, pulp/fuelwood, and forest residues.<br>
- 1.5 degrees - RCP 0 (no CC)
- LTS - RCP 2.6
- NDC - RCP 4.5


In [2]:
RCP = "RCP0"  # "RCP0"  or "RCP26" or "RCP45"

In [3]:
glob_demands = True # True or False

In [4]:
approach = "bottomup" # "topdown" or "bottomup"

## Read the data

In [5]:
import os
import numpy as np
import sys
import requests
import wget

### Read .py class

In [6]:
if glob_demands == True : 
    nameGLOB = "_GLOB"
else :
    nameGLOB = ""

In [7]:
import multiFunctionalOptimization as MFO

In [8]:
from importlib import reload
reload(MFO)

<module 'multiFunctionalOptimization' from 'E:\\MultiForest\\Optimization\\multiforestOptimizationNotebook\\MultiForestDemonstration\\multiFunctionalOptimization.py'>

In [9]:
mfo = MFO.MultiFunctionalOptimization()

'Using CLP'

In [10]:
filename = "rslt_"+RCP+"_Bavaria_Germany_pause_2_V1"+nameGLOB+".zip"
filename

'rslt_RCP0_Bavaria_Germany_pause_2_V1_GLOB.zip'

### Import data

In [11]:
if not filename in os.listdir("."):
    url = "https://syncandshare.lrz.de/dl/fiQMgp5WcrfEe9sJUZptH7Mu/Data/"+filename
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)


In [12]:
### Read data

In [13]:
%%time
mfo.readData(filename,
             sampleRatio=0.2 #If no sample ratio given, the ratio is assumed to be 1
            ) 

'sample size 745/3725(20%)'

Wall time: 3.12 s


In [14]:
mfo.data.columns

Index(['year', 'id', 'regime', 'ecoregion', 'V_strat', 'V_strat_reg',
       'Ve_strat', 'Vm_strat', 'iV_strat', 'pct_coni', 'pct_reg_coni',
       'pct_deci', 'pct_reg_deci', 'spruce_pct', 'spruce_reg_pct', 'fir_pct',
       'fir_reg_pct', 'pine_pct', 'pine_reg_pct', 'larch_pct', 'larch_reg_pct',
       'beech_pct', 'beech_reg_pct', 'oak_pct', 'oak_reg_pct',
       'douglasfir_pct', 'douglasfir_reg_pct', 'hardbrl_pct',
       'hardbrl_reg_pct', 'softbrl_pct', 'softbrl_reg_pct', 'spi', 'spi_reg',
       'maxh', 'maxh_reg', 'spint', 'freq_h_0_50', 'freq_h_50_80',
       'freq_h_80_100', 'cntlayers', 'shan_species', 'shan_species_reg',
       'shan_height', 'volHa_dbhGR10', 'volHa_dbhGR20', 'volHa_dbhGR30',
       'volHa_dbhGR40', 'volHa_dbhGR50', 'volHa_dbhGR60', 'volHa_dead_dbhGR30',
       'volHa_dead_dbhGR40', 'volHa_dead_dbhGR50', 'volHa_dead_dbhGR60',
       'sawntimber', 'industrywood', 'yieldloss', 'remaininstand',
       'represented_area_by_NFIplot', 'region', 'NUTS2', 'clear_c

## Create new columns

In [15]:
if glob_demands == True :     
    glob_sawlogs = mfo.data.groupby('year')['GLOBsawlogs'].apply(lambda x: list(np.unique(x)))
    df = glob_sawlogs.to_frame().reset_index()
    df = df["GLOBsawlogs"]
    flattened_list = [y for x in df for y in x]
    glob_sawlogs = flattened_list
    glob_sawlogs

In [16]:
if glob_demands == True : 
    glob_pulwood = mfo.data.groupby('year')['GLOBpulpwood'].apply(lambda x: (np.unique(x)))
    df = glob_pulwood.to_frame().reset_index()
    df = df["GLOBpulpwood"]
    flattened_list = [y for x in df for y in x]
    glob_pulwood = flattened_list
    glob_pulwood

In [17]:
if glob_demands == True : 
    glob_residues = mfo.data.groupby('year')['GLOBresidues'].apply(lambda x: (np.unique(x)))
    df = glob_residues.to_frame().reset_index()
    df = df["GLOBresidues"]
    flattened_list = [y for x in df for y in x]
    glob_residues = flattened_list
    glob_residues

In [18]:
if glob_demands == True : 
    glob_fuelwood = mfo.data.groupby('year')['GLOBfuelwood'].apply(lambda x: (np.unique(x)))
    df = glob_fuelwood.to_frame().reset_index()
    df = df["GLOBfuelwood"]
    flattened_list = [y for x in df for y in x]
    glob_fuelwood = flattened_list
    glob_fuelwood

In [19]:
if glob_demands == True : 
    zipped_lists = zip(glob_pulwood, glob_fuelwood)
    glod_pulp_fuel = [x + y for (x, y) in zipped_lists]
    glod_pulp_fuel

In [20]:
mfo.data["HarvestedVolume"] = (mfo.data["Ve_strat"].values)*mfo.data["represented_area_by_NFIplot"].values

In [21]:
mfo.data["CarbonBalance"] = (mfo.data["BalanceCarbonTotalYear"].values)*mfo.data["represented_area_by_NFIplot"].values

In [22]:
mfo.data["AnnualIncrement"] = mfo.data["iV_strat"].values*mfo.data["represented_area_by_NFIplot"].values

In [23]:
mfo.data["speciesProfileIndex"] = mfo.data["spi"].values 

In [24]:
mfo.data["VolumeLargeTrees"] = mfo.data["volHa_dbhGR60"].values*mfo.data["represented_area_by_NFIplot"].values

In [25]:
mfo.data["speciesInterminglingIndex"] = mfo.data["spint"].values

In [26]:
mfo.data["ShannonIndexSpecies"] = mfo.data["shan_species"].values

In [27]:
mfo.data["ShannonIndexTreeHeight"] = mfo.data["shan_height"].values

In [28]:
mfo.data["standingVolume"] = mfo.data["V_strat"]*mfo.data["represented_area_by_NFIplot"].values

In [29]:
mfo.data["noNaturalSpecies"] = mfo.data.apply(lambda x: sum([x[name] for name in mfo.data.columns if "pnv" in name]),axis=1)

In [30]:
mfo.data["stateForest"] = mfo.data['landtenure'].apply(lambda x: 1 if x < 1000 else 0)

In [31]:
mfo.data["SawTimber"] = mfo.data["V Sawlogs"].values*mfo.data["represented_area_by_NFIplot"].values

In [32]:
mfo.data["PulpWood"] =mfo.data["V PulpWood"].values*mfo.data["represented_area_by_NFIplot"].values

In [33]:
mfo.data["ForestResidues"] =mfo.data["V HarvestResidues"].values*mfo.data["represented_area_by_NFIplot"].values

In [34]:
mfo.data["Fuelwood"] = mfo.data["fuelwood"].values*mfo.data["represented_area_by_NFIplot"].values

In [35]:
mfo.data["energyProds"] =mfo.data["CFsawlogsToEnergy"].values*mfo.data["represented_area_by_NFIplot"].values

In [36]:
mfo.data["TotalLivingCarbon"] = (mfo.data["CSagbLivingTotal"].values + mfo.data["CSbgbLivingTotal"].values )*mfo.data["represented_area_by_NFIplot"].values

In [37]:
mfo.data["pnvDeviation"] =((mfo.data["pnv_spruce"].values*100/mfo.data["noNaturalSpecies"].values - mfo.data["spruce_pct"].values)**2 + 
                      (mfo.data["pnv_beech"].values*100/mfo.data["noNaturalSpecies"].values - mfo.data["beech_pct"].values)**2 +
                      (mfo.data["pnv_pine"].values*100/mfo.data["noNaturalSpecies"].values - mfo.data["pine_pct"].values)**2+
                      (mfo.data["pnv_fir"].values*100/mfo.data["noNaturalSpecies"].values - mfo.data["fir_pct"].values)**2+
                      (mfo.data["pnv_oak"].values*100/mfo.data["noNaturalSpecies"].values - mfo.data["oak_pct"].values)**2)

C:\Users\torano\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\torano\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [38]:
import numpy as np

In [39]:
mfo.data["pnvDeviation"].replace(np.nan,0,inplace=True)

In [40]:
mfo.data["pnvDeviation"]

5           8483.904800
7           6449.696100
21         15221.355800
23         16714.163600
24          6666.666667
               ...     
1009453     5443.435200
1009465        0.000000
1009469     7776.125000
1009471    14572.982500
1009474    20000.000000
Name: pnvDeviation, Length: 201895, dtype: float64

In [41]:
mfo.data[mfo.regimesEnu].value_counts()

CCF_P1           13410
BAU_FS1          13410
BAU_RR_p2        13410
CCF_P3_p2        13410
BAU_RR_p1        13410
CCF_STATE        13410
NOT              13410
CCF_P3           13410
BAU_1            13410
CCF_P2           13410
BAU_0_p1         13410
CCF_P3_p1        13410
BAU_0_p2         13410
BAU_RR           13410
BAU_0            13410
initial_state      745
Name: regime, dtype: int64

### Add column to restrict SA¶

In [42]:
regimeClassNames = {"regimeClass0name":"CCF","regimeClass1name":"SA"}
regimeClassregimes = {"regimeClass0regimes":["CCF_P3 ","CCF_P3_p1","CCF_P3_p2"],"regimeClass1regimes":["NOT"]}

In [43]:
mfo.addRegimeClassifications(regimeClassNames = regimeClassNames,regimeClassregimes=regimeClassregimes)

In [44]:
mfo.finalizeData(initialRegime="initial_state")

In [45]:
mfo.initialData.head()

,,,ecoregion,V_strat,V_strat_reg,Ve_strat,Vm_strat,iV_strat,pct_coni,pct_reg_coni,pct_deci,pct_reg_deci,...,stateForest,SawTimber,PulpWood,ForestResidues,Fuelwood,energyProds,TotalLivingCarbon,pnvDeviation,CCF_forests,SA_forests
id,year,regime,,,,,,,,,,,,,,,,,,,,,
6,2012,initial_state,9.6.2,138.631058,0.0,0.0,0.0,138.631058,36.0,0.0,64.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,53294.405274,3532.1936,False,False
10,2012,initial_state,9.15.5,228.104820,0.0,0.0,0.0,228.104820,7.0,0.0,93.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,45645.839112,2788.2760,False,False
14,2012,initial_state,9.5.2,182.529020,0.0,0.0,0.0,182.529020,5.0,0.0,95.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,36610.808570,2207.9282,False,False
18,2012,initial_state,9.5.8,395.745146,0.0,0.0,0.0,395.745146,28.0,0.0,72.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,154026.328969,5271.9176,False,False
20,2012,initial_state,9.6.3,185.863275,0.0,0.0,0.0,185.863275,0.0,0.0,100.0,0.0,...,1,0.0,0.0,0.0,0.0,0.0,37487.865513,10000.0000,False,False


In [46]:
mfo.data.head()

ecoregion     V_strat  V_strat_reg  Ve_strat   Vm_strat  \
id year regime                                                             
6  2017 BAU_0        9.6.2  202.927642     0.614593       0.0  16.772962   
        BAU_0_p1     9.6.2  202.912150     0.588503       0.0  16.612431   
        BAU_0_p2     9.6.2  186.253553     0.992174       0.0  27.230879   
        BAU_1        9.6.2  203.502270     0.634451       0.0  16.233236   
        BAU_FS1      9.6.2  201.778571     0.645147       0.0  17.888440   

                   iV_strat  pct_coni  pct_reg_coni  pct_deci  pct_reg_deci  \
id year regime                                                                
6  2017 BAU_0     81.069546      30.0          33.0      70.0          67.0   
        BAU_0_p1  80.893523      30.0          39.0      70.0          61.0   
        BAU_0_p2  74.853373      28.0          36.0      72.0          64.0   
        BAU_1     81.104448      31.0          45.0      69.0          55.0   
        BAU_FS1   81.035954      30.0          38.0      70.0          62.0   

                  ...  Relative_GLOBfuelwood  Relative_AnnualIncrement  \
id year regime    ...                                                    
6  2017 BAU_0     ...               0.001228                  0.000286   
        BAU_0_p1  ...               0.001228                  0.000285   
        BAU_0_p2  ...               0.001228                  0.000264   
        BAU_1     ...               0.001228                  0.000286   
        BAU_FS1   ...               0.001228                  0.000286   

                  Relative_speciesProfileIndex  Relative_VolumeLargeTrees  \
id year regime                                                              
6  2017 BAU_0                         0.002338                        0.0   
        BAU_0_p1                      0.002346                        0.0   
        BAU_0_p2                      0.002333                        0.0   
        BAU_1                         0.002333                        0.0   
        BAU_FS1                       0.002338                        0.0   

                  Relative_speciesInterminglingIndex  \
id year regime                                         
6  2017 BAU_0                               0.002557   
        BAU_0_p1                            0.002557   
        BAU_0_p2                            0.002557   
        BAU_1                               0.002557   
        BAU_FS1                             0.002557   

                  Relative_ShannonIndexSpecies  \
id year regime                                   
6  2017 BAU_0                          0.00262   
        BAU_0_p1                       0.00262   
        BAU_0_p2                       0.00262   
        BAU_1                          0.00262   
        BAU_FS1                        0.00262   

                  Relative_ShannonIndexTreeHeight  Relative_standingVolume  \
id year regime                                                               
6  2017 BAU_0                            0.005693                 0.000715   
        BAU_0_p1                         0.005687                 0.000715   
        BAU_0_p2                         0.005687                 0.000656   
        BAU_1                            0.005683                 0.000717   
        BAU_FS1                          0.005689                 0.000711   

                  Relative_TotalLivingCarbon  Relative_pnvDeviation  
id year regime                                                       
6  2017 BAU_0                       0.000785               0.000421  
        BAU_0_p1                    0.000785               0.000420  
        BAU_0_p2                    0.000723               0.000363  
        BAU_1                       0.000786               0.000438  
        BAU_FS1                     0.000781               0.000418  

[5 rows x 165 columns]

## Objectives

### FS - Forest Strategy

In [47]:
if scenario == 'NFS':
    
    objectives_abs = {
        "HarvestedVolume": ["Sum of extracted volume (maximize, m3/ha) (4)","HarvestedVolume",
                            "max","min","areaWeightedAverage"],
        "AnnualIncrement": ["Annual increment of standing volume (maximimze, m3/ha/5year) (4)","AnnualIncrement",
                            "max","min","areaWeightedAverage"],
        "CteStandingVolume" : ["Standing volume must be maintained ct. (m3) (4)", "Relative_standingVolume",
                             "max", "minYearlyIncrease", "areaWeightedAverage"],
        "EnergyWood" : ["Energy Wood for industry must be maintained ct. (tC) (3)", "energyProds", 
                       "max", "minYearlyIncrease", "sum"],
        "totalCarbonBalance" : ["carbon balance must be constant or maximized (tC) (2)", "CarbonBalance",
                             "max", "min", "sum"]  
    }
    objectives_rel = {
        "speciesProfileIndex": ["Species profile index (Pretzsch) (maximize the mean) (2)","speciesProfileIndex",
                             "max","average","areaWeightedAverage"],                           
        "ShannonIndexSpecies": ["Shannon index of species (maximize the mean) (2)","ShannonIndexSpecies",
                            "max","average","areaWeightedAverage"],
        "RiskStormBB" :["Risk of storms and Bark Beetel (minimize the maximum risk) (2)", "Relative_stormBBRisk", 
                       "min", "max","areaWeightedAverage"],
        "RecreationAsth" : ["Recreation and Aesthetics maximize the mean) (4)", "RecreationAndAesthetics",
                           "max", "average", "areaWeightedAverage"],
        "Biodiveristy": ["Biodiversity fuzzy indicator (mantain ct, evenflow) (4)", "Biodiversity", 
                             "max", "average", "areaWeightedAverage"],
        "CrownCoverage":["Mean Crown coverage must be maintened (minimize change, evenflow) (4)","covered_area_per",
                         "max", "min", "areaWeightedAverage"],
    }

    objectives = {
        **objectives_abs,
        **objectives_rel,
    }

### BDS - Biodiversity Strategy

In [48]:
if scenario == 'BDS':
    objectives_abs = {
    "totalCarbonBalance" : ["carbon balance must be constant or increase (tC) (4)", "CarbonBalance",
                         "max", "average", "areaWeightedAverage"]
    }
    objectives_rel = {
    "relative_LivingCarbon": ["Living carbon stored in Forests must increase by 5% by 2020 (tC - 10% relative to 2007 values) (4)", "Relative_TotalLivingCarbon",
                                "max","targetYearWithSlope","sum", 2020],
    "speciesProfileIndex": ["Species profile index (Pretzsch) (maximum over all years)  (4)","speciesProfileIndex",
                         "max","average","areaWeightedAverage"],  
    "ShannonIndexSpecies": ["Shannon index of species (maximum over all years) (4)","ShannonIndexSpecies",
                        "max","average","areaWeightedAverage"],
    "Biodiveristy": ["Biodiversity fuzzy indicator (maximum over all years) (4)", "Biodiversity", 
                         "max", "average", "areaWeightedAverage"],

    }
    objectives_SA = {
 
     "Ratio_SA_forests": ["Ratio of protected areas (%, SA forests) (1)",
                         "SA_forests",
                         "max","firstYear","areaWeightedAverage"]  
    }
    objectives = {
        **objectives_SA,
        **objectives_abs,
        **objectives_rel,
    }
    objectives.update(
        {"pnvDeviation": ["deviation from potential natural vegetation (min over all years) (4)","pnvDeviation",
        "min","max","areaWeightedAverage"],
        }
    )
 

### BES - Bioeconomy Strategy

In [49]:
if scenario == 'BES':
    
    objectives_abs = {
        
    "energyproducts": ["remaning products for energy use (tC maximize) (4)", "energyProds",
                        "max","min","sum"],
    "PulpWood" : ["wood products for industry (m3 maximize)  (4)", "PulpWood",
                        "max","min","sum"],
    "totalCarbonBalance" : ["carbon balance must be constant or increase (tC) (2)", "CarbonBalance",
                         "max", "average", "sum"],
    "SawTimberProduction" : ["timber production (m3 maximize) (4)", "SawTimber",
                         "max", "min", "sum"],

    }
    objectives_rel = {
        
     "Biodiversity": ["Biodiversity fuzzy indicator (maximize) (1)", "Biodiversity", 
                    "max", "min", "areaWeightedAverage"],
     "CrownCoverage":["Mean Crown coverage must be maintened ct (% cover) (1)","covered_area_per",
                     "max", "min", "areaWeightedAverage"],
    }
    objectives = {
        **objectives_abs,
        **objectives_rel,
    }
    

### GLOBIOM demands bottom-up approach

In [50]:
if glob_demands == True and approach == "bottomup":
    
    objectives_GLOBdemand = {
    
    "pulpwood_demand": ["GLOBIOM pulp wood demand (m3/ha) u. bark", "PulpWood",
                        "max", "periodicTargets", "sum", glod_pulp_fuel],
    "sawlogs_demand":  ["GLOBIOM saw logs demand (m3/ha) u. bark", "SawTimber",
                        "max", "periodicTargets", "sum", glob_sawlogs], 
    "residues_demand": ["GLOBIOM residues demand (m3/ha)", "ForestResidues",
                        "max", "periodicTargets", "sum", glob_residues], 
    }
    objectives.update(objectives_GLOBdemand)

In [51]:
objectives

{'HarvestedVolume': ['Sum of extracted volume (maximize, m3/ha) (4)',
  'HarvestedVolume',
  'max',
  'min',
  'areaWeightedAverage'],
 'AnnualIncrement': ['Annual increment of standing volume (maximimze, m3/ha/5year) (4)',
  'AnnualIncrement',
  'max',
  'min',
  'areaWeightedAverage'],
 'CteStandingVolume': ['Standing volume must be maintained ct. (m3) (4)',
  'Relative_standingVolume',
  'max',
  'minYearlyIncrease',
  'areaWeightedAverage'],
 'EnergyWood': ['Energy Wood for industry must be maintained ct. (tC) (3)',
  'energyProds',
  'max',
  'minYearlyIncrease',
  'sum'],
 'totalCarbonBalance': ['carbon balance must be constant or maximized (tC) (2)',
  'CarbonBalance',
  'max',
  'min',
  'sum'],
 'speciesProfileIndex': ['Species profile index (Pretzsch) (maximize the mean) (2)',
  'speciesProfileIndex',
  'max',
  'average',
  'areaWeightedAverage'],
 'ShannonIndexSpecies': ['Shannon index of species (maximize the mean) (2)',
  'ShannonIndexSpecies',
  'max',
  'average',
  'ar

In [52]:
mfo.defineObjectives(objectives)

'Defining objectives'

'Aggregating stand wise'

100%|██████████| 14/14 [00:27<00:00,  1.98s/it]


'Aggregating year wise'

100%|██████████| 14/14 [00:00<00:00, 2793.68it/s]


'Objectives added'

### GLOBIOM demands top-down approach - option for assortment transfer

In [53]:
if glob_demands == True and approach == "topdown" :
    
    mfo.addGlobiomTargets(
        {
        "log": glob_sawlogs,
        "pulp": glob_pulwood,
        "residues": glob_residues
        },
        {
            #Log is converted primarily into log, and sencondary into pulp; no transferrate, both are volumes under bark
            "SawTimber":
                {"log":[1,"primary"],"pulp":[1,"secondary"], "residues":[1.136,"secondary"]},
                #{"log":[1,"primary"],"pulp":[1,"secondary"]}, 
                                          
            # Pulp is converted primarily into pulp, and sencondary into residues; 
            # with transferrate 1.136, because pulp volume is under bark, residues are over bark (barkfactor = 1.136)  
            "PulpWood":
                {"pulp":[1,"primary"], "residues":[1.136,"secondary"]}, 
            
            "ForestResidues":
                {"residues":[1,"primary"]} #Biomass only to residues
        },
        
        # ----------------
        # by default exactMatching is FALSE
        # ----------------
        # Functionality was implemented for Norway and their nationl policy scenarios
        # NOT required for Cross-scale analysis V2
        exactMatching= False   
        
    )
    
    print("objective function with assortment transfer loaded")

## Define Enabled Constrains

In [54]:
NotCCregimes = [regime for regime in mfo.regimes if "BAU" not in regime] 

In [55]:
NotCCregimes

['CCF_P1', 'CCF_P2', 'CCF_P3', 'CCF_P3_p1', 'CCF_P3_p2', 'CCF_STATE', 'NOT']

In [56]:
constraintTypes = {"NoCC_Prot":["Allowed regimes","NO CleatCut on protected land",NotCCregimes,"protected"],
                   "NoCC_State":["Allowed regimes","NO CleatCut on state forest",NotCCregimes,"stateForest"]}

In [57]:
mfo.defineConstraints(constraintTypes)

## Calculate objective ranges

You can run the following function twice to improve ranges accuracy.

Especially, if some objectives get as lowerbounds 0:s, then it makes sense to reoptimize

In [ ]:

%%time
mfo.calculateObjectiveRanges(debug=True)

'Calculating objective ranges'

  0%|          | 0/14 [00:00<?, ?it/s]

'Optimizing for Sum of extracted volume (maximize, m3/ha) (4)'

'Found an optimal solution in 1 seconds'

'Objective values are:'

...

'GLOBIOM saw logs demand (m3/ha) u. bark'

0.0

'GLOBIOM residues demand (m3/ha)'

0.0

 64%|██████▍   | 9/14 [00:30<00:16,  3.20s/it]

In [ ]:
mfo.objectiveRanges

## Show GUI

In [ ]:
mfo.showGUI()

## Visualization of optimal solution

In [ ]:
regimeAmounts = {regime:0 for regime in mfo.regimes}
for key in mfo.regimesDecision.keys():
    regimeAmounts[key[1]] +=mfo.regimesDecision[key].solution_value()*mfo.standAreas.loc[key[0],"represented_area_by_NFIplot"]/mfo.standAreas["represented_area_by_NFIplot"].sum()

In [ ]:
%pylab notebook

In [ ]:
[val for val in regimeAmounts.values()]

In [ ]:
plt.plot([key for key in regimeAmounts.keys()],[val for val in regimeAmounts.values()])

In [ ]:
plt.bar(range(len(regimeAmounts)), list(regimeAmounts.values()), align='center')
plt.xticks(range(len(regimeAmounts)), list(regimeAmounts.keys()),rotation="vertical")

## Export data as csv
- Solution_alldata contains the optimal regime per stand AND the timely development of indicator values plus all other input columns (represented_are_by_NFIplot, region, NUTS2)
- Solution contains only the selected optimal regime and its share (if multiple regimes per stand are selected)

In [ ]:
if glob_demands == True : 
    approach_glob = "Glob_"+approach
else :
    approach_glob = "NoGlob"
approach_glob

In [ ]:
import pandas as pd
import os

In [ ]:
import os

try:
    os.mkdir("results")
except FileExistsError:
    pass
b = []
c = []
for key in mfo.regimesDecision.keys():
    if mfo.regimesDecision[key].solution_value() > 0:
        b = b+ [(key[0],x*5+2012, key[1]) for x in range(0,21)]
        c = c+ [(key[0],key[1],mfo.regimesDecision[key].solution_value())]
data2b = mfo.data.iloc[mfo.data.index.isin(b)]
data2b.to_csv("./results/"+approach_glob+'_'+scenario+"_"+RCP+"_data.csv")
c1 = pd.DataFrame(c)
c1.to_csv("./results/"+approach_glob+'_'+scenario+"_"+RCP+"_solutions.csv")

## Export objective ranges

In [ ]:
for key in mfo.regimesDecision.keys():
        print(key,mfo.regimesDecision[key].solution_value())

Save as json file

In [ ]:
import json
mfo.objectiveRanges

with open('./objectiveRanges_'+approach_glob+'_'+scenario+'_'+RCP+'.json', 'w') as json_file:
  json.dump(mfo.objectiveRanges, json_file)

Save as CSV.

In [ ]:
import pandas
df = pandas.read_json('./objectiveRanges_'+approach_glob+'_'+scenario+'_'+RCP+'.json')

df.to_csv('./objectiveRanges_'+approach_glob+'_'+scenario+'_'+RCP+'.csv')

## Export objective values
The optimal solution for each objective.

In [ ]:
import os
with open("./objectiveValues_"+approach_glob+"_"+scenario+"_"+RCP+".csv","w") as file:
    delim = ""
    for objName in mfo.objectiveTypes.keys():
        file.write(delim+objName)
        delim = ","
    file.write("\n")
    delim = ""
    for objName in mfo.objectiveTypes.keys():
        file.write(delim+str(mfo.objective[objName].solution_value()))
        delim = ","
    file.write("\n")